# Imports

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import os
import numpy as np
import torchmetrics
import matplotlib.pyplot as plt
import time
import albumentations as A
import seaborn as sns
from train_model import train_model


from PIL import Image
from sklearn.model_selection import train_test_split
from albumentations.pytorch import ToTensorV2
from torchvision.ops import focal_loss
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import confusion_matrix

# Constants

In [2]:
NUM_CLASSES = 20+1 #including background-class
MODEL_PATH = 'model/output/model.pth'
NUM_EPOCHS = 1000
BATCH_SIZE = 16

In [3]:
image_folder = 'data/processed/images/'
mask_folder = 'data/processed/masks/'

In [4]:
# dataset = TacoDataset(image_folder, mask_folder, transform=train_transform)
# labels_count = torch.zeros(NUM_CLASSES).to(device)

# for i, (_, masks) in enumerate(dataset):
#         print(i, end="\r")
#         labels = torch.flatten(masks)
#         labels_count += torch.bincount(labels, minlength=NUM_CLASSES)

In [5]:
# class_inverse_frequencies = 1.0 / labels_count
# class_weights = class_inverse_frequencies / torch.sum(class_inverse_frequencies)
# class_weights[0] = 0

# Trainning model

In [ ]:
model = train_model(MODEL_PATH,
           NUM_CLASSES,
           BATCH_SIZE,
           NUM_EPOCHS,
           image_folder, mask_folder)

Model has been loaded!
Creating dataloaders...
Training model...


  0%|          | 0/1000 [00:00<?, ?it/s]







# Eval model

In [ ]:
model.eval()
example_of_preds = []

preds_arrays = []
mask_arrays = []

with torch.no_grad():
    epoch_iou = 0
    for n, (images, masks) in enumerate(test_dataloader):
        images = images.to(device)
        masks = masks.to(device)
        
        outputs = model(images)['out']
        preds = torch.argmax(outputs, dim=1)

        epoch_iou += IoU(preds, masks)
        
        preds_arrays.append(torch.flatten(masks).cpu().numpy())
        mask_arrays.append(torch.flatten(preds).cpu().numpy())
        
        if n >= len(test_dataloader) - 5:
            images = torch.squeeze(images, dim=0).cpu().numpy().transpose(1, 2, 0)
            preds_image = preds.cpu().numpy().transpose(1, 2, 0)
            mask_image = torch.squeeze(masks, dim=0).cpu().numpy()
            example_of_preds.append([images, preds_image, mask_image])

    epoch_iou /= len(test_dataloader)
    print(f'IoU: {epoch_iou}')

In [ ]:
mask_arrays = np.array(mask_arrays).flatten()
preds_arrays = np.array(preds_arrays).flatten()

In [ ]:
cm = confusion_matrix(mask_arrays, preds_arrays)
cm[0][0] = 0 # ignore 0-class

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=False, cmap="Blues", fmt="d")
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
for images, preds_image, mask_image in example_of_preds:
    fig, axes = plt.subplots(1, 3, figsize=(10, 5))
    axes[0].imshow(images)
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(preds_image)
    axes[1].set_title('Pred Mask')
    axes[1].axis('off')

    axes[2].imshow(mask_image)
    axes[2].set_title('GT Mask')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()